In [7]:
from fastai.collab import *
from fastai.tabular.all import *

# Collaborative filtering

In [8]:
path = untar_data(URLs.ML_100k)
path.ls()

(#23) [Path('/storage/data/ml-100k/u4.test'),Path('/storage/data/ml-100k/u1.base'),Path('/storage/data/ml-100k/u4.base'),Path('/storage/data/ml-100k/u.info'),Path('/storage/data/ml-100k/u2.base'),Path('/storage/data/ml-100k/README'),Path('/storage/data/ml-100k/u.item'),Path('/storage/data/ml-100k/u.data'),Path('/storage/data/ml-100k/u5.test'),Path('/storage/data/ml-100k/u5.base')...]

In [13]:
ratings = pd.read_csv(path/'u.data', sep='\t', header=None, names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# DataLoaders

In [19]:
movies = pd.read_csv(path/'u.item', encoding='latin-1', delimiter='|', header=None, usecols=(0,1), names=('movie', 'title'))
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [20]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [24]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,378,Passion Fish (1992),4
1,178,While You Were Sleeping (1995),4
2,422,Jaws 2 (1978),3
3,75,Kids in the Hall: Brain Candy (1996),4
4,250,Aladdin (1992),5
5,322,"Streetcar Named Desire, A (1951)",4
6,815,Return of the Jedi (1983),5
7,732,Lost Highway (1997),2
8,918,Jurassic Park (1993),3
9,311,Dolores Claiborne (1994),3


In [29]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

n_users, n_movies

(944, 1665)